In [ ]:
import os
import re
import mne
from autoreject import AutoReject
from mne_icalabel import label_components
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
#mne.viz.set_browser_backend('qt')
#mne.set_config('MNE_BROWSER_BACKEND', 'qt')

# Artifact correction

In [ ]:
SUBJECTS = ["R0782_20220411_CrosslingViz_0003", "R0810_20230120_CrosslingViz_0004", "R0811_20230120_CrosslingViz_0005"]

In [ ]:
# Load data

subject = SUBJECTS[0]
file = f'CrossLingViz_EEG/{subject}/{subject}.vhdr'

raw = mne.io.read_raw_brainvision(file, preload=True)

# Filtering

raw_filt = raw.copy().filter(1, 30)
raw_filt.set_channel_types(mapping={'VEOG': 'eog'});

raw_filt.pick_types(eeg=True)
#raw_filt.plot(butterfly=True)

# Bad channel interpolation

#raw_filt.plot()

In [ ]:
#raw_filt.info['bads'].append('FC5') 
#raw_filt.interpolate_bads(reset_bads=True)

# ICA correction

ica = mne.preprocessing.ICA(n_components=25, max_iter='auto', random_state=97)
ica.fit(raw_filt) 
ica

ic_labels = label_components(raw_filt, ica, method="iclabel")
labels = ic_labels["labels"]

exclude_idx = [idx for idx, label in enumerate(labels) if label not in ["brain", "other"]]

ica.exclude = exclude_idx
reconst_raw=raw_filt.copy()
ica.apply(reconst_raw)

reconst_raw = reconst_raw.pick_types(eeg=True)

print("exclude following components for subject", subject, exclude_idx)

In [ ]:
labels

In [ ]:
%notebook inline
ica.plot_components()

In [ ]:
raw_filt.load_data()
ica.plot_sources(raw_filt, show_scrollbars=True);

In [ ]:
ica.exclude = [0, 1, 3, 7]
reconst_raw=raw_filt.copy()
ica.apply(reconst_raw)

reconst_raw = reconst_raw.pick_types(eeg=True)

In [ ]:
exclude_idx

#809: [0, 1, 3, 7]
#810: [0, 1, 2, 3, 20, 24]
#811: [0, 2, 6, 14, 15, 16]

In [ ]:
reconst_raw.plot()

# Epochs (OPTO alignment)

In [ ]:
opto = raw.copy()
opto.pick_channels(['OPTO'])

# OPTO epochs
events =  mne.events_from_annotations(raw);
opto_epochs = mne.Epochs(opto, events[0], tmin=-0.1, tmax=1, baseline=(None, 0.), preload=True)

# Get delays
arr = []
for i in range(len(opto_epochs)):
    d = np.squeeze(opto_epochs[i].get_data())

    bl_M = d[0:50].mean() 
    bl_V = d[0:50].var()

    th = bl_M + (20e3 * bl_V)

    try:
        idx = np.where(d>=th)[0][0]
        shft = (idx - 50) * (1 /  opto_epochs.info['sfreq'] )
        
    except IndexError:
        shft = 0
        
    arr.append(shft)

events = events[0]

events_shft = []

for i in range(1, len(events)):
    events_shft[i:i+1] = mne.event.shift_time_events(events[i:i+1], ids=None, tshift=arr[i-1], sfreq=raw.info['sfreq'])

events_shft = np.vstack(events_shft)

# EEG epochs
Eng = pd.read_excel("G:\내 드라이브\Research\Decoding\PsychopyViz\stimuli_Engv.xlsx")
Kor = pd.read_excel("G:\내 드라이브\Research\Decoding\PsychopyViz\stimuli_Korv.xlsx")

try:
    event_id={'RespYes': 1, 'RespNo': 2} # if all responses = yes
    
    for i in range(64):
        event_id[Eng["word"][i]] = int(Eng["trigger"][i])
    
    for i in range(64):
        event_id[Kor["word"][i]] = int(Kor["trigger"][i])
    
    epochs_aligned = mne.Epochs(reconst_raw, events_shft, event_id, tmin=-0.3, tmax=1, baseline=(None, 0.), preload=True, event_repeated='drop')

except ValueError:
    event_id={'RespYes': 1}
    
    for i in range(64):
        event_id[Eng["word"][i]] = int(Eng["trigger"][i])
    
    for i in range(64):
        event_id[Kor["word"][i]] = int(Kor["trigger"][i])
    
    epochs_aligned = mne.Epochs(reconst_raw, events_shft, event_id, tmin=-0.3, tmax=1, baseline=(None, 0.), preload=True, event_repeated='drop')

# Final cleaning

In [ ]:
# Keep trials w/o responses

epochs_idx=[]

for i in range(len(epochs_aligned)-1):
    if epochs_aligned.events[i+1][2]!=1 and epochs_aligned.events[i+1][2]!=2:
        epochs_idx.append(i)
        
epochs_noresp = epochs_aligned[epochs_idx]

# Autoreject

ar = AutoReject()
ar.fit(epochs_noresp)

epochs_ar, reject_log = ar.transform(epochs_noresp, return_log=True)

epochs_dropped = epochs_ar.copy()
reject_criteria = dict(eeg=100e-6) # 100 µV
epochs_dropped.drop_bad(reject=reject_criteria)
epochs_dropped.plot_drop_log();

print('-------------------------------------')
print(len(epochs_dropped), "epochs left out of", len(epochs_noresp), "epochs")
print('-------------------------------------')

#R0809: dropped 116 (10.2%) 1035
#R0810
#R0811: dropped 31 epochs (2.6%) 1152

# Rereference

epochs_ref = epochs_dropped.copy(); #epochs_ar.copy()
epochs_ref.add_reference_channels('TP9');
epochs_ref.set_eeg_reference(['TP9', 'TP10']);

In [ ]:
epochs_ref.save("G:\내 드라이브\Research\Decoding\EEGViz\R0809_epo.fif", overwrite=True)

# Plots

In [ ]:
import mne
import re
epochs_ref = mne.read_epochs("G:\내 드라이브\Research\Decoding\EEGViz\R0809_auto_epo.fif")

In [ ]:
Eng = pd.read_excel("G:\My Drive\Research\Decoding\PsychopyViz\stimuli_Engv.xlsx")
Kor = pd.read_excel("G:\My Drive\Research\Decoding\PsychopyViz\stimuli_Korv.xlsx")

stim = []
for i in range(64):
    stim.append(Eng["word"][i])
    
for i in range(64):
    stim.append(Kor["word"][i])

In [ ]:
%matplotlib inline

r1 = re.compile("Kor.*goat")
r2 = re.compile("Kor.*duck")
r3 = re.compile("Kor.*swan")
r4 = re.compile("Kor.*lion")

l1 = list(filter(r1.match, stim))
l2 = list(filter(r2.match, stim))
l3 = list(filter(r3.match, stim))
l4 = list(filter(r4.match, stim))

evoked1 = epochs_dropped[l1].average()
evoked2 = epochs_dropped[l2].average()
evoked3 = epochs_dropped[l3].average()
evoked4 = epochs_dropped[l4].average()
evoked1.comment = 'Korean goat'
evoked2.comment = 'Korean duck'
evoked3.comment = 'Korean swan'
evoked4.comment = 'Korean lion'

mne.viz.plot_compare_evokeds([evoked1, evoked2, evoked3, evoked4], picks='Cz', ylim=dict(eeg=[-10, 10]));

In [ ]:
%matplotlib inline

r1 = re.compile("Eng.*goat")
r2 = re.compile("Eng.*duck")
r3 = re.compile("Eng.*swan")
r4 = re.compile("Eng.*lion")

l1 = list(filter(r1.match, stim))
l2 = list(filter(r2.match, stim))
l3 = list(filter(r3.match, stim))
l4 = list(filter(r4.match, stim))

evoked1 = epochs_dropped[l1].average()
evoked2 = epochs_dropped[l2].average()
evoked3 = epochs_dropped[l3].average()
evoked4 = epochs_dropped[l4].average()
evoked1.comment = 'English goat'
evoked2.comment = 'English duck'
evoked3.comment = 'English swan'
evoked4.comment = 'English lion'

mne.viz.plot_compare_evokeds([evoked1, evoked2, evoked3, evoked4], picks='Cz', ylim=dict(eeg=[-10, 10]));

In [ ]:
r1 = re.compile("Eng.*swan")
l1 = list(filter(r1.match, stim))
epochs_ref[l1].plot()

#how to reject the third epoch?

In [ ]:
r1 = re.compile("Eng.*lion")
l1 = list(filter(r1.match, stim))
epochs_ref[l1].plot_image(picks='Cz')

In [ ]:
r1 = re.compile("Eng.*swan")
l1 = list(filter(r1.match, stim))
epochs_dropped[l1].plot_image(picks='Cz')

In [ ]:
r1 = re.compile("Eng.*swan")
l1 = list(filter(r1.match, stim))
epochs_ref[l1].plot_image(picks='Cz')